In [0]:
%pip install transformers
%pip install PyPDF2

In [0]:
%rm -rf BERT_financialReportAnalysis/

In [0]:
!git clone https://github.com/francisliu2/BERT_financialReportAnalysis.git

Cloning into 'BERT_financialReportAnalysis'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 60 (delta 35), reused 41 (delta 16), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [0]:
%cd BERT_financialReportAnalysis/

/content/BERT_financialReportAnalysis


In [0]:
from companytoVector_BERT import *
from corporate_profile_extrator import *

from google.colab import files
import os
import glob

os.mkdir('outputs')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Max number of tokens in one sentence: 512


In [0]:
CP = glob.glob("./data/*.pdf")

In [0]:
model = load_model()

In [0]:
success = []
failed = []
result_temp = [] # for debugging
for i in range(len(CP)):
  name = CP[i].replace('.pdf', '') 
  name = name.replace('./data/', '')
  corporate_profile_text = get_corporate_profile(CP[i])
  corporate_profile_ids = text_tokening(corporate_profile_text, verbose=0)
  if corporate_profile_ids == None:
    failed.append({name, 'sentence is too long'})
  else:
    success.append(name)
    result = company2vector(corporate_profile_ids, model)
    result_temp.append(result)
  np.save('./outputs/'+name, result)

In [0]:
success

['Yongnam Annual Report 2010', 'China Star_Sustainability Report']

In [0]:
!zip -r ./outputs.zip ./outputs
!sleep 2
files.download('outputs.zip')

updating: outputs/ (stored 0%)
updating: outputs/China Star_Sustainability Report.npy (deflated 7%)
updating: outputs/Yongnam Annual Report 2010.npy (deflated 7%)


In [0]:
# Check time needed to run
corporate_profile_text = get_corporate_profile(CP[0])
%time result = company2vector(corporate_profile_text, model)

CPU times: user 29.9 s, sys: 2.69 s, total: 32.6 s
Wall time: 32.7 s
